In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, os
import cv2
%matplotlib inline

INPUT_PATH = 'D:/cloud_kaggle_data/input'
train = pd.read_csv(f'{INPUT_PATH}/train.csv') 
sub = pd.read_csv(f'{INPUT_PATH}/sample_submission.csv')
palet = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]
palet = [(0, 0, 255), (255, 0, 0), (200, 0, 200), (249,255,0)]
from tqdm import tqdm_notebook as tqdm

train['Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().value_counts()
train['label'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train['im_id'] = train['Image_Label'].apply(lambda x: x.split('_')[0])
sub['label'] = sub['Image_Label'].apply(lambda x: x.split('_')[1])
sub['im_id'] = sub['Image_Label'].apply(lambda x: x.split('_')[0])

In [2]:
def get_img(x, folder: str='train_images'):
    data_folder = f"{INPUT_PATH}/{folder}"
    image_path = os.path.join(data_folder, x)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def rle_decode(mask_rle: str = '', shape: tuple = (1400, 2100)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')


def make_mask(df: pd.DataFrame, image_name: str='img.jpg', shape: tuple = (1400, 2100)):
    encoded_masks = df.loc[df['im_id'] == image_name, 'EncodedPixels']
    masks = np.zeros((shape[0], shape[1], 4), dtype=np.float32)
    for idx, label in enumerate(encoded_masks.values):
        if label is not np.nan:
            mask = rle_decode(label, shape=shape)
            masks[:, :, idx] = mask
    return masks


def visualize_train(image_name, oof_df=None, params={'thres': [40, 40, 40, 50], 'minsize': [5000, 5000, 5000, 5000]}, savefolder=None, show=True):
    image = get_img(image_name)
    image =  cv2.resize(image, dsize=(525,350), interpolation=cv2.INTER_LINEAR)
    mask = make_mask(train, image_name)
    mask = cv2.resize(mask, dsize=(525,350), interpolation=cv2.INTER_LINEAR)
    class_dict = {0: 'Fish', 1: 'Flower', 2: 'Gravel', 3: 'Sugar'}
    fig, ax = plt.subplots(figsize=(12, 12))
    for ch in range(4):
        # Draw ground truth masks
        m2 = np.copy(mask[:,:,ch]).astype(np.uint8)
        contours, _ = cv2.findContours(m2, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        for i in range(0, len(contours)):
            cv2.polylines(image, contours[i], True, palet[ch], thickness=2, lineType=1)
        
        if oof_df is not None:
            col = 'rle_'+str(params['thres'][ch])+'_'+str(params['minsize'][ch])
            rle = oof_df.loc[oof_df.im_id==image_name, col].iloc[ch]
            if rle is np.nan: continue
            m2 = rle_decode(rle, (350, 525)).astype(np.uint8)
            contours, _ = cv2.findContours(m2, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            for i in range(0, len(contours)):
                cv2.polylines(image, contours[i], True, palet[ch], thickness=1, lineType=4)
    
    if show:
        ax.imshow(image)
        #plt.show()
                
    if savefolder is not None: plt.savefig(savefolder+image_name+'.jpg', bbox_inches='tight')
    plt.close(fig)

In [3]:
stats = pd.read_csv('D:/cloud_kaggle_data/preds_cloud/ures/stats.csv')

In [4]:
params = {'thres': [40, 40, 40, 40], 'minsize': [5000, 5000, 5000, 5000]}
for img_name in tqdm(stats.iloc[::4]['im_id']):
    visualize_train(img_name, oof_df=stats, params=params, savefolder='D:/cloud_kaggle_data/img_oof/ures_40_5k/', show=False)